In [1]:
from dask.distributed import Client
client = Client()
client

/opt/conda/lib/python3.10/site-packages/distributed/client.py:1391: VersionMismatchWarning: Mismatched versions found

+---------+-----------------+-----------------+-----------------+
| Package | Client          | Scheduler       | Workers         |
+---------+-----------------+-----------------+-----------------+
| python  | 3.10.13.final.0 | 3.10.12.final.0 | 3.10.12.final.0 |
+---------+-----------------+-----------------+-----------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tcp://172.28.0.2:8786' processes=2 threads=4, memory=6.24 GiB>

In [2]:
from distributed.diagnostics.plugin import InstallPlugin, ForwardOutput

def enable_fault_handler():
    import faulthandler
    faulthandler.enable()
    print('enabled fault handler')

enable_fault_handler()
client.run_on_scheduler(enable_fault_handler)

fault_handler_plugin = InstallPlugin(enable_fault_handler, False)
client.register_plugin(fault_handler_plugin)

enabled fault handler


In [3]:
from glob import glob
import os

path  = '/data'

files_to_check = glob(f'{path}/**/*.parquet', recursive=True)
files_to_check = [f's3://test-bucket{f}' for f in files_to_check if os.path.isfile(f)]
files_to_check = files_to_check[:10]

In [6]:
# If call ensure_s3_initialized before the calculation, the error does not occur
# Do not call this cell if you want to reproduce the error

def pa_fix():
    from pyarrow._s3fs import ensure_s3_initialized
    ensure_s3_initialized()

pa_fix_plugin = InstallPlugin(pa_fix, False)
client.register_plugin(pa_fix_plugin)

In [4]:
import dask.bag as db

def check_datetime64_ns(file_path):
    import pyarrow.parquet as pq
    schema = pq.read_schema(file_path)
    columns = schema.pandas_metadata['columns']
    for column in columns:
        if column['numpy_type'] == 'datetime64[ns]':
            return True
    return False

db.from_sequence(files_to_check).filter(check_datetime64_ns).compute()

KilledWorker: Attempted to run task ('filter-check_datetime64_ns-7bf5c6f6953840ce86b221e198a6a24a', 9) on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://172.28.0.4:44097. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

In [ ]:
# This cell will kill the kernel if the ensure_s3_initialized is not called before

import faulthandler
from pyarrow.fs import FileSystem
from concurrent.futures import ThreadPoolExecutor, wait
from pyarrow._s3fs import ensure_s3_initialized

def enable_fault_handler():
    faulthandler.enable()
    print('enabled fault handler')

enable_fault_handler()

# If the ensure_s3_initialized is called before the test, the kernel will not crash
# ensure_s3_initialized()

def test_from_uri():
    try:
        fs, path = FileSystem.from_uri('s3://bucket/file.parquet')
        print("FileSystem.from_uri executed successfully", fs, path)
    except Exception as e:
        print(f"Error occurred: {e}")

num_threads = 2

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(test_from_uri) for _ in range(num_threads)]
    wait(futures)